In [1]:
import numpy as np # Importing numpy for performing array and matrix operations 
import pandas as pd # Importing pandas to read data in dataframe

# Libraries to plot the distribution of classes 
import seaborn as sns
import matplotlib.pyplot as plt
import datetime #I will use this to Convert ISO8601 format to unix time
from matplotlib.pyplot import figure


import time #I will use this to Convert ISO8601 format to unix time

import warnings
warnings.filterwarnings("ignore")

from sklearn import linear_model  #Importing linear models like logistics regression and SVC
from sklearn.preprocessing import LabelEncoder  # To convert every class  into numeric labels 
from sklearn.metrics import f1_score       # to calculate F1 score 
from sklearn.metrics import classification_report # to know every metric
from tqdm import tqdm

from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data1= pd.read_csv('/content/drive/MyDrive/AAIC_Assignment/ytrain_NpxebDC.csv', parse_dates=['timestamp'],
index_col= ['timestamp']) # Reading the csv file in data dataframe 

In [4]:
data, test_data=train_test_split(data1, test_size=0.10, shuffle=False)

In [5]:
data.head()

,S7-T1,S2-T1,S19-T1,S56-T3,S85-T3,S16-T3,S16-T1,S94-T3,S28-T1,S62-T3,S14-T3,S8-T1,S8-T3,S34-T3,S6-T2,S29-T3,S64-T1,S11-T1,S76-T3,S59-T1,S58-T1,S58-T3,S21-T3,S96-T3,S30-T1,S77-T1,S35-T3,S71-T3,S89-T1,S51-T3,S51-T1,S38-T3,S79-T1,S79-T3,S33-T3,S33-T1,S50-T3,S1-T3,S49-T1,S60-T3,...,S71-T2,S36-T2,S79-T2,S16-T2,S94-T2,S9-T2,S21-T1,S92-T2,S11-T2,S51-T2,S60-T2,S19-T2,S62-T2,S31-T2,S96-T2,S27-T2,S23-T2,S28-T2,S37-T2,S39-T2,S68-T2,S56-T2,S41-T2,S35-T2,S7-T2,S75-T2,S20-T2,S95-T2,S5-T2,S2-T2,S47-T2,S65-T2,S32-T2,S21-T2,S13-T2,S97-T2,S25-T1,S25-T2,S98-T2,S99-T2
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-25 00:00:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,Down,Down,Offline,Offline,Offline,Offline,Available,Available,Offline,Passive,Available,Available,Available,Passive,Available,Passive,Available,Passive,Available,Available,Charging,Available,Available,Available,Passive,Available,Available,Available,Available,Offline,Passive,...,Charging,Charging,Available,Offline,Offline,Offline,Available,Available,Offline,Available,Available,Available,Down,Offline,Available,Available,Offline,Available,Passive,Charging,Offline,Down,Offline,Available,Down,Offline,Offline,Available,Charging,Down,Down,Offline,Offline,Available,Available,NaN,NaN,NaN,NaN,NaN
2019-11-25 00:15:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,Down,Down,Offline,Offline,Offline,Offline,Available,Available,Offline,Passive,Available,Available,Available,Passive,Available,Passive,Passive,Passive,Available,Available,Available,Available,Available,Available,Available,Available,Available,Available,Available,Offline,Passive,...,Charging,Charging,Available,Offline,Offline,Offline,Available,Available,Offline,Available,Available,Available,Down,Offline,Available,Available,Offline,Available,Passive,Charging,Offline,Down,Offline,Available,Down,Offline,Offline,Available,Charging,Down,Down,Offline,Offline,Available,Available,NaN,NaN,NaN,NaN,NaN
2019-11-25 00:30:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,Down,Down,Offline,Offline,Offline,Offline,Available,Available,Offline,Passive,Available,Available,Available,Passive,Available,Passive,Passive,Passive,Available,Available,Available,Available,Available,Available,Available,Available,Available,Available,Available,Offline,Passive,...,Charging,Charging,Available,Offline,Offline,Offline,Available,Available,Offline,Available,Available,Available,Down,Offline,Available,Available,Offline,Available,Passive,Passive,Offline,Down,Offline,Available,Down,Offline,Offline,Available,Charging,Down,Down,Offline,Offline,Available,Available,NaN,NaN,NaN,NaN,NaN
2019-11-25 00:45:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,Down,Down,Offline,Offline,Offline,Offline,Available,Available,Offline,Passive,Available,Available,Available,Passive,Available,Passive,Passive,Passive,Available,Available,Offline,Offline,Available,Available,Available,Available,Available,Available,Available,Offline,Passive,...,Charging,Charging,Available,Offline,Offline,Offline,Available,Available,Offline,Offline,Available,Available,Down,Offline,Available,Available,Offline,Available,Passive,Passive,Offline,Down,Offline,Available,Down,Offline,Offline,Available,Charging,Down,Down,Offline,Offline,Available,Available,NaN,NaN,NaN,NaN,NaN
2019-11-25 01:00:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,Down,Down,Offline,Offline,Offline,Offline,Available,Available,Offline,Passive,Available,Available,Available,Passive,Available,Passive,Passive,Passive,Available,Available,Available,Available,Available,Available,Available,Available,Available,Charging,Available,Offline,Passive,...,Charging,Charging,Available,Offline,Offline,Offline,Available,Available,Offline,Available,Available,Available,Down,Offline,Available,Available,Offline,Available,Passive,Passive,Offline,Down,Offline,Available,Down,Offline,Offline,Available,Charging,Down,D

In [6]:
test_data.head()

,S7-T1,S2-T1,S19-T1,S56-T3,S85-T3,S16-T3,S16-T1,S94-T3,S28-T1,S62-T3,S14-T3,S8-T1,S8-T3,S34-T3,S6-T2,S29-T3,S64-T1,S11-T1,S76-T3,S59-T1,S58-T1,S58-T3,S21-T3,S96-T3,S30-T1,S77-T1,S35-T3,S71-T3,S89-T1,S51-T3,S51-T1,S38-T3,S79-T1,S79-T3,S33-T3,S33-T1,S50-T3,S1-T3,S49-T1,S60-T3,...,S71-T2,S36-T2,S79-T2,S16-T2,S94-T2,S9-T2,S21-T1,S92-T2,S11-T2,S51-T2,S60-T2,S19-T2,S62-T2,S31-T2,S96-T2,S27-T2,S23-T2,S28-T2,S37-T2,S39-T2,S68-T2,S56-T2,S41-T2,S35-T2,S7-T2,S75-T2,S20-T2,S95-T2,S5-T2,S2-T2,S47-T2,S65-T2,S32-T2,S21-T2,S13-T2,S97-T2,S25-T1,S25-T2,S98-T2,S99-T2
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-10-02 04:45:00+00:00,Down,Offline,Available,Available,Down,Available,Charging,Available,Available,Down,Offline,Passive,Available,Available,Charging,Available,Available,Available,Passive,Offline,Down,Down,Offline,Available,Passive,Offline,Passive,Available,Available,Available,Charging,Available,Available,Available,Available,Available,Available,Available,Offline,Available,...,Available,Available,Passive,Available,Available,Offline,Offline,Offline,Available,Available,Available,Available,Down,Offline,Available,Passive,Available,Available,Available,Available,Available,Available,Offline,Passive,Down,Passive,Available,Offline,Available,Offline,Down,Offline,Passive,Offline,Available,Passive,Offline,Offline,Available,Available
2020-10-02 05:00:00+00:00,Down,Offline,Available,Available,Down,Available,Charging,Available,Available,Down,Offline,Passive,Available,Available,Charging,Available,Available,Available,Passive,Offline,Down,Down,Offline,Available,Passive,Offline,Passive,Offline,Available,Available,Charging,Available,Available,Available,Available,Available,Available,Available,Offline,Available,...,Offline,Available,Passive,Available,Available,Offline,Offline,Offline,Available,Available,Available,Available,Down,Offline,Available,Passive,Available,Available,Available,Available,Available,Available,Offline,Passive,Down,Passive,Available,Offline,Available,Offline,Down,Offline,Passive,Offline,Available,Passive,Offline,Offline,Available,Available
2020-10-02 05:15:00+00:00,Down,Offline,Available,Available,Down,Available,Charging,Available,Available,Down,Offline,Passive,Available,Available,Charging,Available,Available,Available,Passive,Offline,Down,Down,Offline,Available,Passive,Offline,Passive,Available,Available,Available,Charging,Available,Available,Available,Available,Available,Available,Available,Offline,Available,...,Available,Available,Passive,Available,Available,Offline,Offline,Offline,Available,Available,Available,Available,Down,Offline,Available,Passive,Available,Available,Available,Available,Available,Available,Offline,Passive,Down,Passive,Available,Offline,Available,Offline,Down,Offline,Passive,Offline,Available,Passive,Offline,Offline,Available,Available
2020-10-02 05:30:00+00:00,Down,Offline,Available,Available,Down,Available,Charging,Available,Available,Down,Offline,Passive,Available,Available,Available,Available,Available,Available,Passive,Offline,Down,Down,Offline,Available,Passive,Offline,Passive,Available,Available,Available,Charging,Available,Available,Available,Available,Available,Available,Available,Offline,Available,...,Available,Available,Passive,Available,Available,Available,Offline,Offline,Available,Available,Available,Available,Down,Offline,Available,Passive,Available,Available,Available,Available,Available,Available,Offline,Passive,Down,Passive,Available,Offline,Available,Offline,Down,Offline,Passive,Offline,Available,Passive,Offline,Offline,Available,Available
2020-10-02 05:45:00+00:00,Down,Offline,Available,Available,Down,Available,Charging,Available,Available,Down,Offline,Passive,Available,Available,Available,Available,Available,Available,Passive,Offline,Down,Down,Offline,Available,Passive,Offline,Passive,Available,Available,Available,Charging,Available,Available,Available,Available,Available,Available,Available,Offline,Available,...,Available,Available,Passive,Available,Av

In [7]:
for col in data:    # for every column in the data frame
    if col!='timestamp':
        data[col]=data[col].fillna('Offline')  # Filling the missing class label with Down as explained above
data['S7-T1'].isnull().sum() # counting the NAN values

0

In [11]:
def date_time(data):
# Initializing Empty lists
   
    month=[]
    week=[]
    date=[]
    hr=[]
    minute=[]

# I have made timestamp column as an index of the dataframe for easy plotting operations
    for i in data.index: # For every index of the dataframe
    #print(i)
    #break
    #dt = datetime.datetime.strptime(i, "%Y-%m-%dT%H:%M:%S%z")
        dt=i
    # Appending the month, day, weekday, hour and minute in lists
        
        month.append(dt.month)
        week.append(dt.weekday()) #Monday is 0 and Sunday is 6
        date.append(dt.day)
        hr.append(dt.hour)
        minute.append(dt.minute)
# making X as a data frame 
    return pd.DataFrame(list(zip( month, week, date, hr, minute)), columns =['Month_name', 'Week_number', 'Date', 'Hour', 'Minute'])

In [12]:
X=date_time(data)
X.head()

,Month_name,Week_number,Date,Hour,Minute
0,11,0,25,0,0
1,11,0,25,0,15
2,11,0,25,0,30
3,11,0,25,0,45
4,11,0,25,1,0


In [13]:
X_test=date_time(test_data)
X_test.head()

,Month_name,Week_number,Date,Hour,Minute
0,10,4,2,4,45
1,10,4,2,5,0
2,10,4,2,5,15
3,10,4,2,5,30
4,10,4,2,5,45


In [14]:
# daypart function
def DayPart(hour):
    if hour in [2,3,4,5]: # dawn ==1
        return 1
    elif hour in [6,7,8,9]: # morning ==2
        return 2
    elif hour in [10,11,12,13]: #  noon==3
        return 3
    elif hour in [14,15,16,17]: # Afternoon==4
        return 4
    elif hour in [18,19,20,21]:  # evening ==5
        return 5
    else: return 6    # midnight==6
# utilize it along with apply method


In [15]:
dayparts = X['Hour'].apply(DayPart)
dayparts_test=X_test['Hour'].apply(DayPart)
print(dayparts.shape)
print(dayparts_test.shape)

(28108,)
(3124,)


In [16]:
# one hot encoding
daypart_ohe= pd.get_dummies(dayparts)
daypart_test_ohe=pd.get_dummies(dayparts_test)
print(daypart_ohe.shape)
print(daypart_test_ohe.shape)

(28108, 6)
(3124, 6)


In [17]:
# one hot encoding
week_number_ohe= pd.get_dummies(X['Week_number'])
week_number_test_ohe=pd.get_dummies(X_test['Week_number'])
print(week_number_ohe.shape)
print(week_number_test_ohe.shape)

(28108, 7)
(3124, 7)


In [18]:
# daypart function
def IsWeekEnd(number):
    if number in [5,6]: # if staurday or sunday return 1
        return 1
    else: return 0    # weekdays=6

In [19]:
weekend=X['Week_number'].apply(IsWeekEnd)
weekend_test=X_test['Week_number'].apply(IsWeekEnd)
print(weekend.shape)
print(weekend_test.shape)

(28108,)
(3124,)


In [20]:
x=X.drop(['Week_number'], axis=1)
x_test=X_test.drop(['Week_number'], axis=1)

In [21]:
# merge two sparse matrices: https://stackoverflow.com/a/19710648/4084039 
X_tr = pd.concat([x,daypart_ohe, week_number_ohe, weekend], axis = 1)
X_tr.shape

(28108, 18)

In [22]:
X_te=pd.concat([x_test,daypart_test_ohe, week_number_test_ohe, weekend_test], axis = 1)
X_te.shape

(3124, 18)

### Random Model

In [23]:

def label_encoder(column):
    '''
    This function help us to Encode the class labels of 
    the corresponding terminal into their respective 
    numeric labels
    '''
    if column =='Available': # Available ==0
        return 0
    elif column=='Charging': # charging==1
        return 1
    elif column=='Passive': # Passive==2
        return 2
    elif column=='Offline': # offline==3
        return 3
    else: return 4    # Down==4
# utilize it along with apply method

In [24]:
def weight_func(num_class):
    dct={}
    for num in num_class:
        #if num==0:  # Available
           # dct[0]=1
        if num==1:    #Charging
            dct[1]=9
        if num==2:    # Passive
            dct[2]=10
        if num==3:     # Offline
            dct[3]=3
        if num==4:     # down
            dct[4]=5
    return dct

In [25]:

av_count=0
ch_count=0
pa_count=0
do_count=0
of_count=0
for i in tqdm(range(0,data.shape[1])):
    
    terminal= data.columns[i] # Extracting the column names from the dataframe
    labels=data[terminal]
    for i in labels:
        #print(i)
        if i=='Available':
            av_count+=1
        if i=='Charging':
            ch_count+=1
        if i=='Passive':
            pa_count+=1
        if i=='Down':
            do_count+=1
        if i=='Offline':
            of_count+=1
tot_count=av_count+ch_count+pa_count+do_count+of_count
print(tot_count)
prob_list=[0,0,0,0,0]# order available, passive, charging, offline, down
prob_list[0]=av_count/tot_count
prob_list[1]=pa_count/tot_count
prob_list[2]=ch_count/tot_count
prob_list[3]=of_count/tot_count
prob_list[4]=do_count/tot_count

100%|██████████| 273/273 [00:02<00:00, 104.84it/s]

7673484


Multi layer Perceptron model

In [26]:
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.optimizers import Adagrad
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight 
from keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from keras.layers import Flatten
from keras.layers import Dropout
from numpy import array

In [27]:
class_weight = {0: 1.,
                1: 9.,
               2: 10.,
               3:3.,
               4:5}

In [32]:

f1_state_terminal=[0,0,0,0,0, 0] # order available, passive, charging, offline, down
f1_state_terminal_test=[0,0,0,0,0, 0] # order available, passive, charging, offline, down
label_count=[0,0,0,0,0,0]
for i in tqdm(range(0,data.shape[1])):
  tf.keras.backend.clear_session()
  terminal= data.columns[i] # Extracting the column names from the dataframe
    #num_class=data[terminal].unique() # Unique labels present in the dataframe column
  weekend=data[terminal].apply(label_encoder)
  num_class=weekend.unique()
  num_class.sort()
  weekend_test=test_data[terminal].apply(label_encoder)
  num_class_test=weekend_test.unique()
  num_class_test.sort()
  if len(num_class)>1:
    unit=len(num_class)
    #print(unit)
    y=np.array(weekend).reshape(-1,1) # label encoding class labels
    y_test=np.array(weekend_test).reshape(-1,1)
    y_train1 = to_categorical(y, 5)
    y_test1 = to_categorical(y_test, 5)
    #print(y_train1.shape)
    #print(y_test1.shape)   
            # define model
    model = Sequential()
    model.add(Dense(1024, activation='relu', input_dim=X_tr.shape[1]))
    model.add(Dense(512))
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Dense(units=5, activation='softmax'))
    adam = Adam()
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    
    #class_weights = class_weight.compute_class_weight('balanced',weekend.unique(y) ,data[terminal])
    #class_weights = dict(enumerate(class_weights))
    model.fit(X_tr, y_train1, epochs=20, batch_size=200, verbose=0, class_weight=class_weight)
        
    y_pred=model.predict(X_tr)
    val_predict=np.argmax(y_pred, axis=1) 
    val_target=np.argmax(y_train1, axis=1) 
    state_f1=f1_score(val_target, val_predict, average=None)
        
    y_pred_test=model.predict(X_te)
    val_predict=np.argmax(y_pred_test, axis=1) 
    val_target=np.argmax(y_test1, axis=1) 
    state_f1_test=f1_score(val_target, val_predict, average=None)
        
    idx=0
    for label in num_class:
            #print("more than 1",num_class)
        f1_state_terminal[label]+=state_f1[idx]
        label_count[label]+=1
        idx+=1
            #print('passed from if ')
            
    idx=0
    for label in num_class_test:
        f1_state_terminal_test[label]+=state_f1_test[idx]
        idx+=1
          
  else:
    for label in num_class:
            #print('1 class',num_class)
        f1_state_terminal[label]+=1 # Adding F1 score as 1 to the state where a terminal is in only one mode
        label_count[label]+=1
            #print('passed from else')
        
    for label in num_class_test:
        f1_state_terminal_test[label]+=1 # Adding F1 score as 1 to the state where a terminal is in only one mode
    


100%|██████████| 273/273 [40:30<00:00,  8.90s/it]


In [33]:
#print('The counts of the class labels in train data', label_count)
#print("The Combined F1 scores of Terminals ",f1_state_terminal) # order available, passive, charging, offline, down
for i in range(len(label_count)-1):
    f1_state_terminal[i]/=273
print("The Combined Average F1 scores of train Terminals ",f1_state_terminal) # order available, passive, charging, offline, down

#print("The Combined F1 scores of test Terminals ",f1_state_terminal_test) # order available, passive, charging, offline, down
for i in range(len(label_count)-1):
    f1_state_terminal_test[i]/=273
print("The Combined Average F1 scores of test Terminals ",f1_state_terminal_test) # order available, passive, charging, offline,down

f1_final=0
#print(" The probabilities of the train states",prob_list)
for i in range(len(prob_list)):
    f1_final+=prob_list[i]*f1_state_terminal[i]
print('The final F1 score is', f1_final)

f1_final_test=0

for i in range(len(prob_list)):
    f1_final_test+=prob_list[i]*f1_state_terminal_test[i]
print('The final F1 score is', f1_final_test)

The Combined Average F1 scores of train Terminals  [0.6086979810328907, 0.30881155417912987, 0.42195637968342575, 0.5934121586492463, 0.3539468882364999, 0]
The Combined Average F1 scores of test Terminals  [0.3704549261435706, 0.1173166136347936, 0.0892834791177464, 0.10503660221001507, 0.13422039056801288, 0]
The final F1 score is 0.5511260645525401
The final F1 score is 0.26601179994017726


LSTM based model

In [34]:
X_tr1 = array(X_tr).reshape(X_tr.shape[0],1, X_tr.shape[1])
X_te1 = array(X_te).reshape(X_te.shape[0], 1,X_te.shape[1])

In [37]:

f1_state_terminal=[0,0,0,0,0, 0] # order available, passive, charging, offline, down
f1_state_terminal_test=[0,0,0,0,0, 0] # order available, passive, charging, offline, down
label_count=[0,0,0,0,0,0]

for i in tqdm(range(0,data.shape[1])):
  tf.keras.backend.clear_session()
  
  terminal= data.columns[i] # Extracting the column names from the dataframe
    #num_class=data[terminal].unique() # Unique labels present in the dataframe column
  weekend=data[terminal].apply(label_encoder)
  num_class=weekend.unique()
  num_class.sort()
  weekend_test=test_data[terminal].apply(label_encoder)
  num_class_test=weekend_test.unique()
  num_class_test.sort()
  if len(num_class)>1:
    unit=len(num_class)
    #print(unit)
    y=np.array(weekend).reshape(-1,1) # label encoding class labels
    y_test=np.array(weekend_test).reshape(-1,1)
    y_train1 = to_categorical(y, 5)
    y_test1 = to_categorical(y_test, 5)
    #print(y_train1.shape)
    #print(y_test1.shape)   
            # define model
    model = Sequential()
    model.add(LSTM(128, activation='relu', input_shape=( 1, 18)))

    model.add(Dense(200, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(50))
    model.add(Dense(5, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    model.fit(X_tr1, y_train1, epochs=15, batch_size=50, verbose=0)
    
    #class_weights = class_weight.compute_class_weight('balanced',weekend.unique(y) ,data[terminal])
    #class_weights = dict(enumerate(class_weights))
    #model.fit(X_tr, y_train1, epochs=10, batch_size=100, validation_data=(X_te,y_test1), verbose=0)
        
    y_pred=model.predict(X_tr1)
    val_predict=np.argmax(y_pred, axis=1) 
    val_target=np.argmax(y_train1, axis=1) 
    state_f1=f1_score(val_target, val_predict, average=None)
        
    y_pred_test=model.predict(X_te1)
    val_predict=np.argmax(y_pred_test, axis=1) 
    val_target=np.argmax(y_test1, axis=1) 
    state_f1_test=f1_score(val_target, val_predict, average=None)
        
    idx=0
    for label in num_class:
            #print("more than 1",num_class)
        f1_state_terminal[label]+=state_f1[idx]
        label_count[label]+=1
        idx+=1
            #print('passed from if ')
            
    idx=0
    for label in num_class_test:
        f1_state_terminal_test[label]+=state_f1_test[idx]
        idx+=1
          
  else:
    for label in num_class:
            #print('1 class',num_class)
        f1_state_terminal[label]+=1 # Adding F1 score as 1 to the state where a terminal is in only one mode
        label_count[label]+=1
            #print('passed from else')
        
    for label in num_class_test:
        f1_state_terminal_test[label]+=1 # Adding F1 score as 1 to the state where a terminal is in only one mode
    



  0%|          | 0/273 [00:00<?, ?it/s]



  1%|          | 2/273 [00:51<1:55:56, 25.67s/it]



  1%|▏         | 4/273 [01:32<1:48:26, 24.19s/it]



  2%|▏         | 6/273 [02:14<1:43:20, 23.22s/it]



  3%|▎         | 7/273 [02:57<2:08:18, 28.94s/it]

...



 99%|█████████▊| 269/273 [2:48:01<02:42, 40.72s/it]



 99%|█████████▉| 270/273 [2:48:46<02:05, 41.93s/it]



 99%|█████████▉| 271/273 [2:49:29<01:24, 42.27s/it]



100%|█████████▉| 272/273 [2:50:11<00:42, 42.09s/it]



100%|██████████| 273/273 [2:50:54<00:00, 37.56s/it]


In [38]:
#print('The counts of the class labels in train data', label_count)
#print("The Combined F1 scores of Terminals ",f1_state_terminal) # order available, passive, charging, offline, down
for i in range(len(label_count)-1):
    f1_state_terminal[i]/=273
print("The Combined Average F1 scores of train Terminals ",f1_state_terminal) # order available, passive, charging, offline, down

#print("The Combined F1 scores of test Terminals ",f1_state_terminal_test) # order available, passive, charging, offline, down
for i in range(len(label_count)-1):
    f1_state_terminal_test[i]/=273
print("The Combined Average F1 scores of test Terminals ",f1_state_terminal_test) # order available, passive, charging, offline,down

f1_final=0
#print(" The probabilities of the train states",prob_list)
for i in range(len(prob_list)):
    f1_final+=prob_list[i]*f1_state_terminal[i]
print('The final F1 score is', f1_final)

f1_final_test=0

for i in range(len(prob_list)):
    f1_final_test+=prob_list[i]*f1_state_terminal_test[i]
print('The final F1 score is', f1_final_test)

The Combined Average F1 scores of train Terminals  [0.8720513882949165, 0.31633619886168374, 0.5925047964240113, 0.7257569571129644, 0.3873240599111841, 0]
The Combined Average F1 scores of test Terminals  [0.5774743703325854, 0.04662304343503901, 0.05760982685344029, 0.09004004658722199, 0.1367597166364328, 0]
The final F1 score is 0.7461846980316141
The final F1 score is 0.380231904805819
